## Revenue Analysis GameDev

Discription: The main goal was grouping and calculation of certain metrics for further calculation in Tableau. I used CTE, left join, window functions (LAG, LEAD), UNION ALL(for creating the new column and assigning the type of metrics). Link to the visualization: https://public.tableau.com/views/RevenueAnalysis_16999719117190/RevenueAnalysis?:language=en-US&:display_count=n&:origin=viz_share_link

WITH revenue AS (
SELECT
	gp.user_id,
	gp.game_name,
	date_trunc('month',
	gp.payment_date) AS payment_month,
	gpu."language",
	gpu.has_older_device_model,
	gpu.age,
	sum(revenue_amount_usd) AS total_revenue
FROM
	project.games_payments gp
LEFT JOIN project.games_paid_users gpu ON
	gpu.user_id = gp.user_id
GROUP BY
	1,
	2,
	3,
	4,
	5,
	6
),
month_condition AS (
SELECT
	*,
	date(payment_month - INTERVAL '1' MONTH) AS previous_calendar_month,
	date(payment_month + INTERVAL '1' MONTH) AS next_calendar_month,
	LAG(total_revenue) OVER(PARTITION BY user_id
ORDER BY
	payment_month) AS previous_paid_month_revenue,
	LAG(payment_month) OVER(PARTITION BY user_id
ORDER BY
	payment_month) AS previous_paid_month,
	LEAD(payment_month) OVER(PARTITION BY user_id
ORDER BY
	payment_month) AS next_paid_month
FROM
	revenue r
),
metrics_type AS(
--mrr
SELECT
	payment_month,
		user_id,
		game_name,
		total_revenue AS revenue_amount,
		'mrr' AS metrics_type,
		"language",
		has_older_device_model,
		age
FROM
	month_condition
UNION ALL
--expansion_mrr:
SELECT
	payment_month,
		user_id,
		game_name,
		total_revenue AS revenue_amount,
		'expansion_mrr' AS metrics_type,
		"language",
		has_older_device_model,
		age
FROM
	month_condition
WHERE
	previous_paid_month = previous_calendar_month
	AND total_revenue > previous_paid_month_revenue
UNION ALL
--contraction_mrr:
SELECT
	payment_month,
		user_id,
		game_name,
		-total_revenue AS revenue_amount,
		'contraction_mrr' AS metrics_type,
		"language",
		has_older_device_model,
		age
FROM
	month_condition
WHERE
	previous_paid_month = previous_calendar_month
	AND total_revenue < previous_paid_month_revenue
UNION ALL
--churned_users and churned_revenue
SELECT
	next_calendar_month,
		user_id,
		game_name,
		total_revenue AS revenue_amount,
		'churned' AS metrics_type,
		"language",
		has_older_device_model,
		age
FROM
	month_condition
WHERE
	next_paid_month IS NULL
	OR next_paid_month != next_calendar_month
)	
SELECT
	*
FROM
	metrics_type
